In [160]:
import numpy as np   #importing libraries
import simpy 
import queue
import random as rd
import matplotlib 
from matplotlib import pyplot as plt

In [161]:
global SZ,CP,RM
SZ=[]
CP=[]
RM=[]
Lambda = 1
mu = 1.25
t = 0   #system clock
t_lambda = 0 #poisson random variable used to generate next service time
t_mu = 0  #poisson random variable used to generate next departure time
u=rd.random()
wait=0
R=1000
transmission_time=0
waitlist=[]
servicetime=[]

class Packet(object):
    def __init__(self, time, id, mobileID, seeed):
        
        self.time=time                                                
        self.id=id
        self.mobileID=mobileID   
        #random.seed(seeed)
        self.siz=random.randint(200,400) #seed 1 op 2, seed 2 op 
        SZ.append(round(self.siz,2))
        #print(seeed)
        #random.seed(seeed)
        self.cpu=random.randint(20,90) #seed 1 op 2, seed 2 op 
        CP.append(round(self.cpu,2))
        #random.seed(seeed)
        self.ram=random.randint(1, 32) #seed 1 op 3, seed 2 op 1, seed 3 op 4
        RM.append(round(self.ram,2))

global Tsla, Nsla, Gsla, Cost_sla, Cg,Ct, Cn          #Global variables for reward function
global count
global X #x no of requests
global TP #y throughput
global Z #z response time
global QN #w queue size 
TP=[]
Z=[]
QN=[]
X=[] 
count=int(0)
L=queue.Queue(maxsize=6)



class mobile(object): #mobile class with an ID              
    def __init__(self, ID):
        self.ID=ID
        self.pot=int(1)
        self.block=int(0)
        global L
        
    #generates packets and sends via out_pipe
    def packet_generator(self, numPackets, env, out_pipe, j): 
        global queue_size
        queue_size=0
        task=j
        for i in range(numPackets):
            yield env.timeout(random.expovariate(1/5))
            print("sending task {} of mobile {} at time {}".format(task, self.ID, env.now))
            p=Packet(env.now, j, self.ID, self.pot)
            self.pot+=1
            if(self.pot==4):
                self.pot=1
            yield out_pipe.put(p)
            item = yield out_pipe.get()
            print("hellowwwww", item)
            print("jk")
            print("got",item.time)



In [162]:
def FOG(env, server, service_time, in_pipe):
    
     msg = yield in_pipe.get()
     transmission_time = (msg.siz/R)
     print("transmits task {} to need time {}".format(msg.id,transmission_time))
     yield env.timeout(transmission_time)
     a=env.now
     print("arrived task {} at time {}".format(msg.id,a))
     with server.request() as req:
         yield req
         s=env.now
         wait=s-a
         print("starting service task {} at time {}".format(msg.id,s))
         print("task {} waiting time is {}".format(msg.id,wait))
         waitlist.append(wait)
         yield env.timeout(service_time)
         print("finish service task {} at time {}".format(msg.id,env.now))

In [163]:
env1 = simpy.Environment()    
server = simpy.Resource(env1, capacity=1)

pipe1 = simpy.Store(env1)
pipe2 = simpy.Store(env1)
pipe3 = simpy.Store(env1)

ms1=mobile(1)
ms2=mobile(2)
ms3=mobile(3)

for i in range(5):
    u=rd.random()
    t_mu=-np.log(u)/mu  
    env1.process(ms1.packet_generator(1, env1, pipe1, i))
    env1.process(FOG(env1, server, t_mu, pipe1))
    
    #servicetime.append(t_mu)
    
    env1.process(ms2.packet_generator(1, env1, pipe2, i))
    env1.process(FOG(env1, server, t_mu, pipe2))
    
env1.run()


sending task 3 of mobile 2 at time 0.732907315988135
transmits task 3 to need time 0.331
sending task 3 of mobile 1 at time 0.8337090342294093
transmits task 3 to need time 0.319
arrived task 3 at time 1.063907315988135
starting service task 3 at time 1.063907315988135
task 3 waiting time is 0.0
arrived task 3 at time 1.1527090342294093
finish service task 3 at time 1.3420080993061587
starting service task 3 at time 1.3420080993061587
task 3 waiting time is 0.18929906507674943
finish service task 3 at time 1.6201088826241823
sending task 1 of mobile 1 at time 1.6645468357355222
transmits task 1 to need time 0.395
arrived task 1 at time 2.059546835735522
starting service task 1 at time 2.059546835735522
task 1 waiting time is 0.0
finish service task 1 at time 2.3031421890257984
sending task 0 of mobile 1 at time 2.399572664031754
transmits task 0 to need time 0.365
arrived task 0 at time 2.7645726640317543
starting service task 0 at time 2.7645726640317543
task 0 waiting time is 0.0
fin